# 03 - model1 - baseline

In [137]:
import pandas as pd
import seaborn as sns
import string
import warnings
import matplotlib.pyplot as plt
import pickle
import numpy as np
import nltk
import spacy 
from spacy.lang.en import English
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")

#### 1. Read in data

In [2]:
df = pd.read_csv('/home/jupyter/sb-entity-classification/data/data.csv')
df.columns = ['class','name']

####  2. Train, Validation, Test split

In [3]:
train, test = train_test_split(df, test_size = 0.2, random_state = 42, shuffle = True)

#### 3. Create features

##### 3.1. Simple statistical features already explored in EDA

In [4]:
def create_statistical_features(df):
    df_features = df.copy()

    df_features['num_chars'] = df['name'].str.len()
    df_features['num_words'] = df_features['name'].apply(lambda x: len(x.split()))

    count = lambda l1,l2: sum([1 for x in l1 if x in l2]) 
    df_features['num_punctuations']= df_features['name'].apply(lambda x: count(x,set(string.punctuation)))
    df_features['num_ascii']= df_features['name'].apply(lambda x: count(x,set(string.ascii_letters)))
    df_features['num_whitespace']= df_features['name'].apply(lambda x: count(x,set(string.whitespace)))
    df_features['num_digits']= df_features['name'].apply(lambda x: count(x,set(string.digits)))
    df_features['num_nonascii'] = df_features['num_chars']-df_features['num_punctuations']-df_features['num_digits']-df_features['num_ascii']- df_features['num_whitespace']
    df_features = df_features.drop(['num_whitespace', 'num_ascii'], axis = 1)
    
    return df_features

In [5]:
train_statistical_features = create_statistical_features(train)
all_cols = train_statistical_features.columns.tolist()
statistical_feature_cols = [x for x in all_cols if x not in ['class', 'name']]
train_statistical_features = train_statistical_features[statistical_feature_cols]

##### 3.2 keyword in brackets also explored in EDA

In [6]:
# based on common sense -
hand_code_keywords = ['company']+['musician', 'singer', 'writer', 'artist', 'author']\
+ ['footballer', 'cricketer', 'football', 'baseball', 'rugby', 'hockey'] +\
['politician'] +['ship'] + ['crater', 'river'] + ['horse'] +['plant']\
+['album', 'EP', 'soundtrack'] +['film'] +['novel', 'magazine', 'book', 'journal', 'play', 'comics', 'newspaper', 'manga']

In [7]:
def create_keyword_features(df):
    df_features = df.copy()
    df_features['within_brackets'] = df['name'].str.extract('.*\((.*)\).*')

    df_features_with_brackets = df_features[['within_brackets']].dropna(axis = 0)
    for keyword in hand_code_keywords:
        df_features_with_brackets['has_{}'.format(keyword)] = df_features_with_brackets['within_brackets']\
                                                                .apply(lambda x: keyword in x.lower() )

    df_features = df_features.drop('within_brackets', axis = 1)
    df_features_with_brackets = df_features_with_brackets.drop('within_brackets', axis = 1)

    df_features_with_brackets.replace({True:1, False:0}, inplace = True)
    df_features = df_features.merge(df_features_with_brackets, left_index = True, right_index = True, how = 'left')
    df_features.fillna(0, inplace = True)
    
    return df_features

In [8]:
train_keyword_features = create_keyword_features(train)
all_cols = train_keyword_features.columns.tolist()
keyword_feature_cols = [x for x in all_cols if x not in ['class', 'name']]
train_keyword_features = train_keyword_features[keyword_feature_cols]

##### 3.3 NER explored in EDA

In [70]:
nlp = spacy.load("en_core_web_sm")

In [119]:
# use nlp pipeline to paralellize
def get_ner_labels(texts_list):
    labels_list = []
    for doc in nlp.pipe(texts_list, disable=["parser"]):
        labels = [ ent.label_ for ent in doc.ents]
        labels_list.append(labels)
    return labels_list

In [120]:
def create_ner_features(df, NER_labels = None):
    
    df_features = df.copy()

    df_features['NER_labels'] = get_ner_labels(df_features['name'].tolist())
    df_features_with_NER = df_features[df_features['NER_labels'].apply(lambda x: len(x) !=0)][['NER_labels']]
    
    if NER_labels == None:
        all_NER_labels = set(np.concatenate(df_features_with_NER['NER_labels'].tolist()))
    else:
        all_NER_labels = NER_labels
        
    for ner in all_NER_labels:
        df_features_with_NER['ner_{}'.format(ner)] = df_features_with_NER['NER_labels'].apply(lambda x: ner in x )

    df_features = df_features.drop('NER_labels', axis = 1)
    df_features_with_NER = df_features_with_NER.drop('NER_labels', axis = 1)

    df_features_with_NER.replace({True:1, False:0}, inplace = True)
    df_features = df_features.merge(df_features_with_NER, left_index = True, right_index = True, how = 'left')
    df_features.fillna(0, inplace = True)
    
    return df_features, all_NER_labels

In [127]:
%%time
train_ner_features, NER_labels_trained = create_ner_features(train)
all_cols = train_ner_features.columns.tolist()
ner_feature_cols = [x for x in all_cols if x not in ['class', 'name']]
train_ner_features = train_ner_features[ner_feature_cols]

CPU times: user 3min 58s, sys: 378 ms, total: 3min 59s
Wall time: 3min 59s


#### 4. Prepare feature set

In [128]:
train = train.merge(train_statistical_features,left_index = True, right_index = True, how = 'left')\
.merge(train_keyword_features,left_index = True, right_index = True, how = 'left')\
.merge(train_ner_features,left_index = True, right_index = True, how = 'left')

In [129]:
X = train.drop(['name','class'], axis = 1)
y = train['class']

#### 5. Train xgboost

In [130]:
model = XGBClassifier(nthread = -1)
model.fit(X, y)

[18:47:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, nthread=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

#### 6. Evaluate model

In [131]:
# test = test.sample(frac = 0.1)

test_statistical_features = create_statistical_features(test)[statistical_feature_cols]
test_keyword_features = create_keyword_features(test)[keyword_feature_cols]

test_ner_features, _ = create_ner_features(test, NER_labels_trained)
test_ner_features = test_ner_features[ner_feature_cols]

In [132]:
test = test.merge(test_statistical_features,left_index = True, right_index = True, how = 'left')\
            .merge(test_keyword_features,left_index = True, right_index = True, how = 'left')\
            .merge(test_ner_features,left_index = True, right_index = True, how = 'left')

In [133]:
test.fillna(0, inplace = True)

In [134]:
X_test = test.drop(['name','class'], axis = 1)
y_test = test['class']

In [135]:
y_pred = model.predict(X_test)

In [136]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 43.63%


In [138]:
print(classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           1      0.297     0.302     0.299      7982
           2      0.461     0.686     0.551      8034
           3      0.383     0.171     0.237      8004
           4      0.307     0.656     0.418      7907
           5      0.457     0.337     0.388      8152
           6      0.839     0.683     0.753      7981
           7      0.365     0.253     0.299      7944
           8      0.430     0.499     0.462      7959
           9      0.405     0.445     0.424      8103
          10      0.364     0.209     0.265      7913
          11      0.375     0.612     0.465      8011
          12      0.638     0.495     0.557      7917
          13      0.526     0.487     0.506      8031
          14      0.657     0.213     0.322      5820

    accuracy                          0.436    109758
   macro avg      0.465     0.432     0.425    109758
weighted avg      0.461     0.436     0.427    109758



In [139]:
classes_list = pd.read_csv('/home/jupyter/sb-entity-classification/data/classes.txt', header = None)
classes_list['class'] = classes_list.index
classes_list.columns = ['class_name', 'class']
classes_list['class'] = classes_list['class'] + 1  # based on information provided in the brief
classes_list

,class_name,class
0,Company,1
1,EducationalInstitution,2
2,Artist,3
3,Athlete,4
4,OfficeHolder,5
5,MeanOfTransportation,6
6,Building,7
7,NaturalPlace,8
8,Village,9
9,Animal,10
